In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')


In [ ]:
train

In [ ]:
# Converting Target column to int values and splitting targets
label_dict = {val:idx for idx, val in enumerate(sorted(train['target'].unique()))}
train['target'] = train['target'].map(label_dict)

target = train['target']
train.drop(['target'], inplace=True, axis=1)
train.drop(['id'], inplace=True, axis=1)
test.drop(['id'], inplace=True, axis=1)

In [ ]:
print(train.shape)
print(target.shape)

In [ ]:
# Checking Null values
# pd.options.display.max_rows = 4000
# train.isnull().values.any|()
# target.isnull().values.any()
test.isnull().values.any()

In [ ]:
# inspecting features in train data
train.describe()

In [ ]:
# PCA
from sklearn.decomposition import PCA
pca = PCA().fit(train)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

fig, ax = plt.subplots()
xi = np.arange(1, 76, step=1)
y1 = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y1, marker='o', linestyle='--', color='g')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 76, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 53)
train = pca.fit_transform(train)
test = pca.transform(test)

In [ ]:
# Normalizing data
from sklearn import preprocessing
train = preprocessing.normalize(train)
test = preprocessing.normalize(test)

In [ ]:

import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, KFold


In [ ]:
print(target.shape)
print(train.shape)

In [ ]:
# Converting target to categorial values
from keras.utils import to_categorical
# train = train.values
target = target.values
target =  to_categorical(target)
target

In [ ]:
# Splitting Train data for training and validation
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.3,stratify=target)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


In [ ]:
# Deep Neural Network 

In [ ]:
# Hyper parameters
from keras import optimizers

# Learning rate
learning_rate = 0.01

# Epoch  
epochs = 10

# Batch size
batch_size = 100

# Optimizer
opt = optimizers.Adam(lr=learning_rate)

In [ ]:
# Sequential Model
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import activations
from tensorflow.keras.layers import Dropout, BatchNormalization

model = Sequential()
model.add(Dense(700, activation='tanh', input_dim = 53))
model.add(BatchNormalization())
Dropout(0.3)

model.add(Dense(350, activation='tanh'))
model.add(BatchNormalization())
Dropout(0.2)

model.add(Dense(120, activation='tanh'))
model.add(BatchNormalization())
Dropout(0.2)

model.add(Dense(9, activation='softmax'))
# use_bias=False

In [ ]:
# Complie the model

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    epochs=epochs,
    verbose = 1
)

In [ ]:
a = history.history.get('val_accuracy')[-1]
print('Accuracy on Validation data: %.2f' % (a*100))

In [ ]:
plt.plot(history.history['accuracy'], 'r-', label = 'accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], 'c-',  label='accuracy' )
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Valid'], loc='upper right')
plt.show


In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4','Class_5','Class_6', 'Class_7', 'Class_8','Class_9']] = model.predict(test)

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv(f'submission.csv',index=False)